In [1]:
# Imports
import torch
from dask_jobqueue import SLURMCluster
from distributed import Client
import Regression_estimation_testing_Normal
import Regression_estimation_testing_Exponential
import Regression_estimation_testing_HS
import Regression_estimation_testing_Cauchy
import Regression_estimation_testing_IG
import Regression_estimation_testing_Binary

In [2]:
seeds = 100
nsamples = [200,1000]

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="20GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10092",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
futures = []
for seed in range(seeds):
    for n in nsamples:
        f1 = client.submit(Regression_estimation_testing_Normal.run_experiment, seed,n)
        f2 = client.submit(Regression_estimation_testing_Exponential.run_experiment, seed,n)
        f3 = client.submit(Regression_estimation_testing_HS.run_experiment, seed,n)
        f4 = client.submit(Regression_estimation_testing_Cauchy.run_experiment, seed,n)
        f5 = client.submit(Regression_estimation_testing_IG.run_experiment, seed,n)
        f6 = client.submit(Regression_estimation_testing_Binary.run_experiment, seed,n)
        futures += [f1,f2,f3,f4,f5,f6]

In [5]:
results = client.gather(futures)

In [6]:
client.close()
cluster.close()

2024-12-28 18:36:21,009 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [7]:
torch.save(f = "Regression_estimation_results.pt", obj = results)